<h1  align="center"><b> MODELO MA </b></h1>

`Objetivo Geral:` Importar a série temporal transformada dos dados pluviométricos do município de São Paulo e realizar a modelagem MA (Moving Average -> Média Móvel) para previsão.

`Dados:` Os dados foram transformados na pasta [Transformação e Decomposição](../[3]%20Transformação%20e%20Decomposição%20-%20Projeto%20Chuva/) do Projeto Chuva.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Importando Dados e Escolhendo o Modelo </b></h2>
&emsp;&emsp; O modelo MA é outra parte do modelo ARIMA. Ele é uma combinação linear dos erros de previsão. Para escolher o melhor modelo utilizaremos o critério de informação de Akaike (AIC) que quanto menor melhor.

`Observação:` O modelo MA(q) é equivalente a ARIMA(0,0,q).



In [3]:
série_chuva = pd.read_csv('../[3] Transformação e Decomposição - Projeto Chuva/Série Transformada - Chuva Mensal.csv', sep = ';', index_col = 0)
série_chuva = pd.Series(série_chuva['Chuva Mensal (mm)'])
série_chuva.index = pd.date_range('1985', periods = len(série_chuva), freq = 'M')

In [4]:
from statsmodels.tsa.arima.model import ARIMA

# Vendo os valores de AIC para os modelos MA(1) até MA(9). Não convém ir além disso, pois o com o aumento da ordem o tempo de processamento aumenta muito (sem grandes ganhos de AIC)
for i in range(1, 10): 
    modelo_MA = ARIMA(série_chuva, order = (i, 0, 0))
    resultado_MA = modelo_MA.fit()
    print(f'ARIMA(0, 0, {i}): {resultado_MA.aic}')

ARIMA(0, 0, 1): 1577.9031699742663
ARIMA(0, 0, 2): 1578.8492539900853
ARIMA(0, 0, 3): 1548.8664265682196
ARIMA(0, 0, 4): 1515.7737788215243
ARIMA(0, 0, 5): 1468.316854938198
ARIMA(0, 0, 6): 1422.5237584036386
ARIMA(0, 0, 7): 1397.2722987981438
ARIMA(0, 0, 8): 1398.101926577467
ARIMA(0, 0, 9): 1400.0629195660995


In [5]:
modelo_MA = ARIMA(série_chuva, order = (0, 0, 7)) # O modelo ARIMA(0, 0, 7) foi o que apresentou o menor AIC. Obs: Esse modelo tem AIC menor que o modelo ARINA(7, 0, 0) que criamos anteriormente.
resultado_MA = modelo_MA.fit() # Treinando o modelo
print(resultado_MA.summary()) # Sumário do modelo

                               SARIMAX Results                                
Dep. Variable:      Chuva Mensal (mm)   No. Observations:                  456
Model:                 ARIMA(0, 0, 7)   Log Likelihood                -734.598
Date:                Sun, 24 Dec 2023   AIC                           1487.195
Time:                        20:07:20   BIC                           1524.298
Sample:                    01-31-1985   HQIC                          1501.811
                         - 12-31-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7805      0.028    173.067      0.000       4.726       4.835
ma.L1          0.2691      0.046      5.842      0.000       0.179       0.359
ma.L2          0.1999      0.046      4.348      0.0

<h2 align="center"><b> Análise de Resíduos </b></h2>

In [ ]:
resíduo = resultado_MA.resid
resíduo.plot() # Plotando o resíduo
plt.show()

![MA_Residuos](./Gráficos/MA_Residuos.png)

### `Normalidade dos Resíduos:`

In [7]:
import scipy.stats as stats
import seaborn as sns

In [ ]:
stats.probplot(resíduo, dist = 'norm', plot = plt)
plt.title('Normal QQ Plot - Resíduos')
plt.show()

sns.histplot(resíduo, kde = True)
plt.title('Histograma - Resíduos')
plt.show()

![Normal QQ Plot - Resíduos](./Gráficos/Normal%20QQ%20Plot%20-%20Resíduos.png)
![Histograma - Resíduos](./Gráficos/Histograma%20-%20Resíduos.png)

In [11]:
def teste_shapiro(série):
    e, p = stats.shapiro(série)
    print(f'Estatística de Teste = {e}')
    print(f'p-valor = {p}')
    print(f'Resultado: {"É Normal" if (p > 0.05) else "Não é Normal"}')

teste_shapiro(resíduo)

Estatística de Teste = 0.9807164669036865
p-valor = 9.450029210711364e-06
Resultado: Não é Normal


&emsp;&emsp; Esse modelo ficou pior que só com a transformação por raiz cúbica. Então é esperado que a previsão não seja tão boa.

### `Autocorrelação dos Resíduos:`

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(resíduo, lags = 20)
plt.title('Função de Autocorrelação - Resíduos')
plt.show()

plot_pacf(resíduo, lags = 20)
plt.title('Função de Autocorrelação Parcial - Resíduos')
plt.show()

![Função de Autocorrelação - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20-%20Resíduos.png)
![Função de Autocorrelação Parcial - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20Parcial%20-%20Resíduos.png)

&emsp;&emsp; As funções de autocorrelação mostram que os resíduos possuem correlação com os valores passados, o que é um péssimo sinal.

<h2 align="center"><b> Previsão </b></h2>

In [12]:
tamanho_série = len(série_chuva) # Tamanho da série
previsão = resultado_MA.predict(start = tamanho_série, end = tamanho_série + 11) # Previsão para os próximos 12 meses

In [14]:
previsão2 = resultado_MA.forecast(steps = 12) # Método alternativo para previsão

In [ ]:
plt.plot(série_chuva, label = 'Série Original')
plt.plot(série_chuva - resíduo, label = 'Resíduo')
plt.plot(previsão, label = 'Previsão')
plt.title('Previsão - Modelo MA(7)')
plt.legend(loc = 'best')
plt.show()

![Previsão - Modelo MA(7)](./Gráficos/Previsão%20-%20Modelo%20MA(7).png)

&emsp;&emsp; Finalmente chegamos na previsão, mas esses valores passaram por uma transformação por raiz cúbica, então precisamos elevar os valores ao cubo para obter a previsão real.

<h2 align="center"><b> Finalização </b></h2>

In [13]:
previsão_final = previsão ** 3
previsão_final.name = 'Previsão MA(7)'
previsão_final.to_csv('Previsão MA - Chuva Mensal.csv', sep = ';', header = True) # Salvando a previsão em um arquivo csv
display(previsão_final)

2023-01-31    188.898467
2023-02-28    173.029719
2023-03-31    115.217687
2023-04-30    100.401526
2023-05-31    106.712045
2023-06-30     98.267777
2023-07-31    100.778937
2023-08-31    109.251069
2023-09-30    109.251069
2023-10-31    109.251069
2023-11-30    109.251069
2023-12-31    109.251069
Freq: M, Name: Previsão MA(7), dtype: float64